In [0]:
# !pip install holidays

In [0]:
# =======================================================================
# PASSO 1: CONFIGURAÇÃO E PARÂMETROS
# =======================================================================
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, IntegerType, DateType
import holidays

# -- Parâmetros Configuráveis --
data_inicio = "2015-01-01"
data_fim = "2035-12-31"
nome_tabela_final = "production.refined.dim_tempo"

print(f"Gerando dimensão de tempo de {data_inicio} a {data_fim}.")

# =======================================================================
# PASSO 2: GERAR A SEQUÊNCIA DE DATAS BASE
# =======================================================================
# A função 'sequence' é a forma mais performática de gerar um intervalo de datas no Spark.
df_base_dates = spark.sql(f"SELECT explode(sequence(to_date('{data_inicio}'), to_date('{data_fim}'), interval 1 day)) AS data_completa")

# =======================================================================
# PASSO 3: FUNÇÃO UDF PARA IDENTIFICAR FERIADOS
# =======================================================================

def identificar_feriado(data):
    """
    Usa a biblioteca 'holidays' para verificar se uma data é feriado nacional no Brasil.
    Retorna o nome do feriado ou 'Não é Feriado'.
    """
    feriados_br = holidays.country_holidays('BR')
    return feriados_br.get(data, "Não é Feriado")

# Registra a função como uma UDF (User-Defined Function)
feriado_udf = F.udf(identificar_feriado, StringType())

# =======================================================================
# PASSO 4: ENRIQUECER O DATAFRAME COM TODOS OS ATRIBUTOS E MÉTRICAS
# =======================================================================
print("Calculando atributos da dimensão de tempo...")

df_dim_tempo = (
    df_base_dates
    # --- Chave Primária ---
    .withColumn("id_tempo", F.date_format(F.col("data_completa"), "yyyyMMdd").cast(IntegerType()))

    # --- Componentes de Ano ---
    .withColumn("ano", F.year(F.col("data_completa")))

    # --- Componentes de Semestre ---
    .withColumn("semestre", F.when(F.quarter(F.col("data_completa")).isin([1, 2]), 1).otherwise(2))
    .withColumn("nome_semestre", F.when(F.col("semestre") == 1, "1º Semestre").otherwise("2º Semestre"))

    # --- Componentes de Trimestre ---
    .withColumn("trimestre", F.quarter(F.col("data_completa")))
    .withColumn("nome_trimestre", F.concat(F.col("trimestre").cast(StringType()), F.lit("º Trimestre")))

    # --- Componentes de Mês ---
    .withColumn("mes", F.month(F.col("data_completa")))
    .withColumn("nome_mes", F.date_format(F.col("data_completa"), "MMMM")) # 'MMMM' retorna o nome por extenso
    .withColumn("nome_mes_abrev", F.date_format(F.col("data_completa"), "MMM")) # 'MMM' retorna a abreviação

    # --- Componentes de Semana ---
    .withColumn("semana_do_ano", F.weekofyear(F.col("data_completa")))
    .withColumn("dia_da_semana", F.dayofweek(F.col("data_completa"))) # Domingo=1, Sábado=7
    .withColumn("nome_dia_da_semana", F.date_format(F.col("data_completa"), "EEEE")) # 'EEEE' retorna o nome por extenso

    # --- Componentes de Dia ---
    .withColumn("dia_do_ano", F.dayofyear(F.col("data_completa")))
    .withColumn("dia_do_mes", F.dayofmonth(F.col("data_completa")))
    
    # --- Feriados (usando a UDF) ---
    .withColumn("nome_feriado", feriado_udf(F.col("data_completa")))

    # --- Flags (Sinalizadores) ---
    .withColumn("flag_fim_de_semana", F.when(F.col("dia_da_semana").isin([1, 7]), 1).otherwise(0).cast(IntegerType()))
    .withColumn("flag_feriado", F.when(F.col("nome_feriado") != "Não é Feriado", 1).otherwise(0).cast(IntegerType()))
    .withColumn("flag_ultimo_dia_mes", F.when(F.col("data_completa") == F.last_day(F.col("data_completa")), 1).otherwise(0).cast(IntegerType()))
)

# --- Reordenando as colunas para uma melhor organização ---
ordem_colunas = [
    "id_tempo", "data_completa", "ano", "semestre", "nome_semestre", "trimestre", "nome_trimestre",
    "mes", "nome_mes", "nome_mes_abrev", "semana_do_ano", "dia_da_semana", "nome_dia_da_semana",
    "dia_do_ano", "dia_do_mes", "nome_feriado", "flag_fim_de_semana", "flag_feriado",
    "flag_ultimo_dia_mes"
]
df_dim_tempo_final = df_dim_tempo.select(ordem_colunas)

print("Cálculos concluídos. Amostra da dimensão:")
display(df_dim_tempo_final.limit(10))


# =======================================================================
# PASSO 5: SALVAR A TABELA DELTA FINAL
# =======================================================================
print(f"\nSalvando tabela final em: {nome_tabela_final}")

(
    df_dim_tempo_final.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(nome_tabela_final)
)

print("Tabela 'dim_tempo' salva com sucesso!")
# Otimização final
spark.sql(f"OPTIMIZE {nome_tabela_final} ZORDER BY (data_completa)")
print("Otimização concluída.")